In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset

In [2]:
training_epochs = 100
learning_rate = 1e-3
batch_size = 32

In [3]:
filename = 'noise_removed_dataset.npz'
dataset = np.load(filename)

In [4]:
inputs_train = dataset['inputs_train']
inputs_test  = dataset['inputs_test']
labels_train = dataset['labels_train']
labels_test  = dataset['labels_test']

In [5]:
class Custom_dataset(Dataset):
    
    def __init__(self, inputs, labels):
        self.inputs = torch.from_numpy(inputs).float()
        self.labels = torch.from_numpy(labels).long()
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        inputs = self.inputs[index]
        labels = self.labels[index]
        return inputs, labels

In [6]:
train_set = Custom_dataset(inputs= inputs_train,
                           labels= labels_train)
test_set  = Custom_dataset(inputs= inputs_test,
                           labels= labels_test)

In [7]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader  = DataLoader(dataset=test_set,
                          batch_size=batch_size,
                          shuffle=False,
                          drop_last=False)

In [8]:
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    # todo: rewrite when torchscript supports any
    def any_requires_grad(self, input):
        # type: (List[Tensor]) -> bool
        for tensor in input:
            if tensor.requires_grad:
                return True
        return False


    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input):  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features

In [9]:
class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)

In [10]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

In [11]:
class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0.3, num_classes=11, memory_efficient=False):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(2, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [12]:
GPU_NUM = 6
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Currernt cuda device ', torch.cuda.current_device())

if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Currernt cuda device  6
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [13]:
def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    net = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    return net


def densenet_custom(pretrained=False, progress=True, **kwargs):
    return _densenet('densenet_custom', 32, (4, 4, 4), 64, pretrained, progress, **kwargs)

In [14]:
net = densenet_custom().to(device)

In [15]:
net

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [17]:
def acc_check(Net, dataset, epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataset:
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = Net(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc = (100 * correct / total)
        return acc

In [18]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, labels)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    test_acc  = acc_check(net, test_loader, epoch)
    train_acc = acc_check(net, train_loader, epoch)
    print('[Epoch : {:>4}]'.format(epoch + 1, avg_cost))
    print('cost = {:>.7} \t Acc_Test  : {:>.3} \t Acc_Train : {:>.3}'.format(avg_cost, test_acc, train_acc))
print('Learning finished')

[Epoch :    1]
cost = 1.612375 	 Acc_Test  : 59.0 	 Acc_Train : 69.9
[Epoch :    2]
cost = 0.975832 	 Acc_Test  : 79.0 	 Acc_Train : 76.0
[Epoch :    3]
cost = 0.7090138 	 Acc_Test  : 87.0 	 Acc_Train : 86.3
[Epoch :    4]
cost = 0.5948341 	 Acc_Test  : 80.0 	 Acc_Train : 80.1
[Epoch :    5]
cost = 0.5870334 	 Acc_Test  : 90.0 	 Acc_Train : 87.9
[Epoch :    6]
cost = 0.551919 	 Acc_Test  : 77.0 	 Acc_Train : 78.9
[Epoch :    7]
cost = 0.4672896 	 Acc_Test  : 85.5 	 Acc_Train : 88.6
[Epoch :    8]
cost = 0.405177 	 Acc_Test  : 88.0 	 Acc_Train : 86.3
[Epoch :    9]
cost = 0.3486934 	 Acc_Test  : 92.0 	 Acc_Train : 90.5
[Epoch :   10]
cost = 0.316048 	 Acc_Test  : 91.5 	 Acc_Train : 89.1
[Epoch :   11]
cost = 0.348053 	 Acc_Test  : 88.0 	 Acc_Train : 87.2
[Epoch :   12]
cost = 0.3096175 	 Acc_Test  : 85.5 	 Acc_Train : 84.1
[Epoch :   13]
cost = 0.3635437 	 Acc_Test  : 86.5 	 Acc_Train : 87.9
[Epoch :   14]
cost = 0.2616387 	 Acc_Test  : 91.5 	 Acc_Train : 90.5
[Epoch :   15]
cost = 0.27

In [19]:
dataset = np.load('contest_noise_removed.npz')
inputs_contest = dataset['inputs_contest']
labels_contest = dataset['labels_contest']

In [20]:
contest_set = Custom_dataset(inputs= inputs_contest,
                             labels= labels_contest)

In [21]:
contest_loader  = DataLoader(dataset=contest_set,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False)

In [22]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    predicted_testing = torch.empty((1,), dtype=torch.int32).to(device)
    labels_testing = torch.empty((1,), dtype=torch.int32).to(device)
    
    for data in contest_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()
        
        predicted_testing = torch.cat([predicted_testing, predicted.to(device, dtype=torch.int32)], dim=0)
        labels_testing = torch.cat([labels_testing, labels.to(device, dtype=torch.int32)], dim=0)
        
        testing_stack = torch.stack((labels_testing[1:], predicted_testing[1:]), dim=1)

print('Accuracy of the network on the train images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the train images: 37 %


In [23]:
cmt_testing = torch.zeros(11, 11, dtype=torch.int32)

for p in testing_stack:
    tl, pl = p.tolist()
    cmt_testing[tl, pl] += 1

In [24]:
cmt_testing.numpy()

array([[ 7,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [43,  9,  5,  5,  0,  4,  0,  1,  0,  0,  8],
       [11, 10, 10,  9,  0,  2,  2,  1,  0,  0,  1],
       [ 4,  1,  5, 26,  2,  2,  1,  0,  0,  0,  1],
       [ 0,  0,  0, 16, 18,  3,  0,  2,  0,  0,  0],
       [ 1,  1,  0,  3,  4, 25, 10,  0,  0,  0,  2],
       [ 0,  2,  0,  2,  1,  4, 24,  5,  0,  1,  1],
       [ 1,  2,  0,  3,  0,  1, 12, 21,  2,  5,  0],
       [ 0,  4,  0,  0,  0,  1,  1, 22, 19,  8,  2],
       [ 0,  0,  0,  0,  0,  1,  0,  4,  2,  3,  2],
       [ 9,  3,  0, 11,  2, 11, 13,  6,  1,  7, 24]], dtype=int32)

In [25]:
state = {
    'Epoch': epoch,
    'State_dict': net.state_dict(),
    'optimizer': optimizer.state_dict()
}

In [26]:
torch.save(state, 'DenseNet.pt')